In [2]:
# Import des bibliothèques nécessaires
import pandas as pd
import numpy as np
import seaborn as sns
import matplotlib.pyplot as plt
from sklearn.preprocessing import LabelEncoder


# Chargement du jeu de données
def charger_donnees(chemin):
    """
    Charge le jeu de données à partir du chemin spécifié.
    
    Args:
    chemin (str): Chemin du fichier CSV contenant les données.
    
    Returns:
    pandas.DataFrame: Le jeu de données chargé.
    """
    data = pd.read_csv('BankChurners.csv')
    return data

# Exploration des données
def exploration_donnees(data):
    """
    Effectue une analyse exploratoire des données.
    
    Args:
    data (pandas.DataFrame): Le jeu de données à explorer.
    """
    # Affichage des premières lignes du jeu de données
    print("Aperçu des premières lignes du jeu de données :")
    print(data.head())

    # Infos sur les types de données et les valeurs manquantes
    print("\nInformations sur les types de données et les valeurs manquantes :")
    print(data.info())

    # Statistiques descriptives pour les variables numériques
    print("\nStatistiques descriptives pour les variables numériques :")
    print(data.describe())

    # Visualisation des distributions des variables numériques
    print("\nVisualisation des distributions des variables numériques :")
    for column in data.select_dtypes(include='number').columns:
        sns.histplot(data[column], kde=True)
        plt.title(f'Distribution de {column}')
        plt.show()

    # Visualisation des distributions des variables catégorielles
    print("\nVisualisation des distributions des variables catégorielles :")
    for column in data.select_dtypes(include='object').columns:
        sns.countplot(data[column])
        plt.title(f'Distribution de {column}')
        plt.show()

# Nettoyage des données
def nettoyage_donnees(data):
    """
    Nettoie les données en gérant les valeurs manquantes, les valeurs aberrantes, etc.
    
    Args:
    data (pandas.DataFrame): Le jeu de données à nettoyer.
    
    Returns:
    pandas.DataFrame: Le jeu de données nettoyé.
    """
    #Drop des colonnes non utiles
    data = data.drop(["CLIENTNUM", 
              "Naive_Bayes_Classifier_Attrition_Flag_Card_Category_Contacts_Count_12_mon_Dependent_count_Education_Level_Months_Inactive_12_mon_1", 
              "Naive_Bayes_Classifier_Attrition_Flag_Card_Category_Contacts_Count_12_mon_Dependent_count_Education_Level_Months_Inactive_12_mon_2"
             ], axis=1)
    
    # Autres opérations de nettoyage...

    return data

# Encodage des variables catégorielles
def encodage_categories(data):
    """
    Encode les variables catégorielles en variables numériques.
    
    Args:
    data (pandas.DataFrame): Le jeu de données avec des variables catégorielles.
    
    Returns:
    pandas.DataFrame: Le jeu de données avec les variables catégorielles encodées.
    """
    original_data = data.copy()
    original_data['Gender'] = original_data['Gender'].map({'M':1, 'F':O})
    le = LabelEncoder()
    data['class'] = le.fit_transform(data['Attrition_Flag'])
    data = data.drop('Attrition_Flag', axis=1)
    return data

# Division des données en ensembles d'entraînement et de test
def division_donnees(data, target_column, test_size=0.2, random_state=42):
    """
    Divise les données en ensembles d'entraînement et de test.
    
    Args:
    data (pandas.DataFrame): Le jeu de données à diviser.
    target_column (str): Le nom de la colonne cible à prédire.
    test_size (float): La proportion des données à inclure dans l'ensemble de test (par défaut : 0.2).
    random_state (int): La seed pour la reproductibilité (par défaut : 42).
    
    Returns:
    tuple: Un tuple contenant les ensembles d'entraînement et de test (X_train, X_test, y_train, y_test).
    """
    from sklearn.model_selection import train_test_split
    
    X = data.drop(columns=[target_column])
    y = data[target_column]
    
    X_train, X_test, y_train, y_test = train_test_split(X, y, test_size=test_size, random_state=random_state)
    
    return X_train, X_test, y_train, y_test


In [3]:
# Import des bibliothèques nécessaires
from sklearn.linear_model import LogisticRegression
from sklearn.ensemble import RandomForestClassifier
from sklearn.metrics import accuracy_score, precision_score, recall_score, f1_score, roc_auc_score

# Construction et évaluation des modèles
def construire_evaluer_modeles(X_train, X_test, y_train, y_test):
    """
    Construction et évaluation des modèles d'apprentissage automatique.
    
    Args:
    X_train (pandas.DataFrame): Caractéristiques de l'ensemble d'entraînement.
    X_test (pandas.DataFrame): Caractéristiques de l'ensemble de test.
    y_train (pandas.Series): Étiquettes de l'ensemble d'entraînement.
    y_test (pandas.Series): Étiquettes de l'ensemble de test.
    """
    # Initialisation des modèles
    models = {
        "Régression Logistique": LogisticRegression(),
        "Forêt aléatoire": RandomForestClassifier()
    }

    # Entraînement et évaluation des modèles
    for model_name, model in models.items():
        print(f"Entraînement du modèle : {model_name}")
        model.fit(X_train, y_train)
        y_pred = model.predict(X_test)

        # Calcul des métriques de performance
        accuracy = accuracy_score(y_test, y_pred)
        precision = precision_score(y_test, y_pred)
        recall = recall_score(y_test, y_pred)
        f1 = f1_score(y_test, y_pred)
        roc_auc = roc_auc_score(y_test, y_pred)

        # Affichage des métriques
        print(f"Métriques de performance pour le modèle {model_name}:")
        print(f"Accuracy : {accuracy}")
        print(f"Precision : {precision}")
        print(f"Recall : {recall}")
        print(f"F1-score : {f1}")
        print(f"AUC-ROC : {roc_auc}")
        print("\n")


In [4]:
# Import des bibliothèques nécessaires
from sklearn.cluster import KMeans
from sklearn.metrics import silhouette_score

# Recherche du nombre optimal de clusters
def trouver_nombre_clusters(X, max_clusters=10):
    """
    Trouve le nombre optimal de clusters en utilisant la méthode du coude et le score de silhouette.
    
    Args:
    X (pandas.DataFrame): Données à clusteriser.
    max_clusters (int): Nombre maximal de clusters à considérer (par défaut : 10).
    
    Returns:
    int: Nombre optimal de clusters.
    """
    # Initialisation des listes pour stocker les résultats
    elbow_scores = []
    silhouette_scores = []

    # Calcul des scores pour différentes valeurs de k
    for k in range(2, max_clusters+1):
        kmeans = KMeans(n_clusters=k, random_state=42)
        kmeans.fit(X)
        elbow_scores.append(kmeans.inertia_)
        silhouette_scores.append(silhouette_score(X, kmeans.labels_))

    # Affichage des résultats
    plt.figure(figsize=(12, 6))
    plt.subplot(1, 2, 1)
    plt.plot(range(2, max_clusters+1), elbow_scores, marker='o')
    plt.title("Méthode du Coude")
    plt.xlabel("Nombre de Clusters")
    plt.ylabel("Somme des Carrés des Distances")

    plt.subplot(1, 2, 2)
    plt.plot(range(2, max_clusters+1), silhouette_scores, marker='o')
    plt.title("Score de Silhouette")
    plt.xlabel("Nombre de Clusters")
    plt.ylabel("Score de Silhouette")

    plt.show()

    # Sélection du nombre optimal de clusters en utilisant le score de silhouette
    optimal_clusters = silhouette_scores.index(max(silhouette_scores)) + 2
    print(f"Nombre optimal de clusters selon le score de silhouette : {optimal_clusters}")

    return optimal_clusters

# Clustering avec K-means
def clustering_kmeans(X, n_clusters):
    """
    Effectue le clustering des données en utilisant l'algorithme K-means.
    
    Args:
    X (pandas.DataFrame): Données à clusteriser.
    n_clusters (int): Nombre de clusters.
    
    Returns:
    pandas.Series: Étiquettes de clusters pour chaque point de données.
    """
    kmeans = KMeans(n_clusters=n_clusters, random_state=42)
    labels = kmeans.fit_predict(X)
    return labels


In [8]:
import dash
from dash import dcc, html
from dash.dependencies import Input, Output
import plotly.express as px

# Initialisation de l'application Dash
app = dash.Dash(__name__)

# Layout du tableau de bord
app.layout = html.Div([
    html.H1("Tableau de Bord - Insights sur les Clients Bancaires"),
    
    # Ajouter des éléments interactifs (graphiques, filtres, etc.)
])

# Fonction pour mettre à jour les éléments interactifs en fonction des sélections de l'utilisateur
@app.callback(
    # Définir les sorties à mettre à jour
    [Output(component_id='output-element-id', component_property='children')],
    
    # Définir les entrées qui déclenchent la mise à jour
    [Input(component_id='input-element-id', component_property='value')]
    
)
def update_output_div(input_value):
    """
    Fonction pour mettre à jour les éléments interactifs en fonction des sélections de l'utilisateur.
    
    Args:
    input_value: La valeur sélectionnée par l'utilisateur.
    
    Returns:
    str: Contenu mis à jour pour l'élément de sortie.
    """
    # Logique de mise à jour basée sur la valeur d'entrée
    # Cette fonction sera adaptée en fonction des éléments interactifs ajoutés dans le layout du tableau de bord
    
    return 'Output: {}'.format(input_value)

# Exécution de l'application Dash
if __name__ == '__main__':
    app.run_server(debug=True)
